## 👉 START HERE: How to use this notebook

# Step 2: Create tools for your Agent

<todo>


**Important note:** Throughout this notebook, we indicate which cell's code you:
- ✅✏️ should customize - these cells contain code & config with business logic that you should edit to meet your requirements & tune quality.
- 🚫✏️ should not customize - these cells contain boilerplate code required to load/save/execute your Agent

*Cells that don't require customization still need to be run!  You CAN change these cells, but if this is the first time using this notebook, we suggest not doing so.*

### 🚫✏️ Install Python libraries

You do not need to modify this cell unless you need additional Python packages in your Agent.

In [0]:
# %pip install -qqqq -U -r requirements.txt
# # Restart to load the packages into the Python environment
# dbutils.library.restartPython()

### 🚫✏️ Connect to Databricks

If running locally in an IDE using Databricks Connect, connect the Spark client & configure MLflow to use Databricks Managed MLflow.  If this running in a Databricks Notebook, these values are already set.

In [0]:
from mlflow.utils import databricks_utils as du

if not du.is_in_databricks_notebook():
    from databricks.connect import DatabricksSession
    import os

    spark = DatabricksSession.builder.getOrCreate()
    os.environ["MLFLOW_TRACKING_URI"] = "databricks"

### 🚫✏️ Load the Agent's UC storage locations; set up MLflow experiment

This notebook uses the UC model, MLflow Experiment, and Evaluation Set that you specified in the [Agent setup](02_agent_setup.ipynb) notebook.

In [0]:
from cookbook.config.shared.agent_storage_location import AgentStorageConfig
from cookbook.databricks_utils import get_mlflow_experiment_url
from cookbook.config import load_serializable_config_from_yaml_file
import mlflow 

# Load the Agent's storage locations
agent_storage_config: AgentStorageConfig= load_serializable_config_from_yaml_file("./configs/agent_storage_config.yaml")

# Show the Agent's storage locations
agent_storage_config.pretty_print()

# set the MLflow experiment
experiment_info = mlflow.set_experiment(agent_storage_config.mlflow_experiment_name)
# If running in a local IDE, set the MLflow experiment name as an environment variable
os.environ["MLFLOW_EXPERIMENT_NAME"] = agent_storage_config.mlflow_experiment_name

print(f"View the MLflow Experiment `{agent_storage_config.mlflow_experiment_name}` at {get_mlflow_experiment_url(experiment_info.experiment_id)}")

# create tools

- we will store all tools in the `user_tools` folder
- first, create a local function & test it with pytest
- then, deploy it as a UC tool & test it with pytest
- then, add the tool to the Agent 

always reload the tool's code

In [0]:
%load_ext autoreload
%autoreload 3

## lets do an example of a simple, but fake tool that translates old to new SKUs.

1, create the python function that will become your UC function.  you need to annotate the function with docstrings & type hints - these are used to create the tool's metadata in UC.

In [0]:
%%writefile tools/sample_tool.py

def sku_sample_translator(old_sku: str) -> str:
    """
    Translates a pre-2024 SKU formatted as "OLD-XXX-YYYY" to the new SKU format "NEW-YYYY-XXX".

    Args:
        old_sku (str): The old SKU in the format "OLD-XXX-YYYY".

    Returns:
        str: The new SKU in the format "NEW-YYYY-XXX".

    Raises:
        ValueError: If the SKU format is invalid, providing specific error details.
    """
    import re

    if not isinstance(old_sku, str):
        raise ValueError("SKU must be a string")

    # Normalize input by removing extra whitespace and converting to uppercase
    old_sku = old_sku.strip().upper()

    # Define the regex pattern for the old SKU format
    pattern = r"^OLD-([A-Z]{3})-(\d{4})$"

    # Match the old SKU against the pattern
    match = re.match(pattern, old_sku)
    if not match:
        if not old_sku.startswith("OLD-"):
            raise ValueError("SKU must start with 'OLD-'")
        if not re.match(r"^OLD-[A-Z]{3}-\d{4}$", old_sku):
            raise ValueError(
                "SKU format must be 'OLD-XXX-YYYY' where X is a letter and Y is a digit"
            )
        raise ValueError("Invalid SKU format")

    # Extract the letter code and numeric part
    letter_code, numeric_part = match.groups()

    # Additional validation for numeric part
    if not (1 <= int(numeric_part) <= 9999):
        raise ValueError("Numeric part must be between 0001 and 9999")

    # Construct the new SKU
    new_sku = f"NEW-{numeric_part}-{letter_code}"
    return new_sku


Now, let's import the tool and test it locally

In [0]:
from tools.sample_tool import sku_sample_translator

sku_sample_translator("OLD-XXX-1234")

now, lets write some pyTest unit tests for the tool - these are just samples, you will need to write your own

In [0]:
%%writefile tools/test_sample_tool.py
import pytest
from tools.sample_tool import sku_sample_translator



def test_valid_sku_translation():
    """Test successful SKU translation with valid input."""
    assert sku_sample_translator("OLD-ABC-1234") == "NEW-1234-ABC"
    assert sku_sample_translator("OLD-XYZ-0001") == "NEW-0001-XYZ"
    assert sku_sample_translator("old-def-5678") == "NEW-5678-DEF"  # Test case insensitivity


def test_whitespace_handling():
    """Test that the function handles extra whitespace correctly."""
    assert sku_sample_translator("  OLD-ABC-1234  ") == "NEW-1234-ABC"
    assert sku_sample_translator("\tOLD-ABC-1234\n") == "NEW-1234-ABC"


def test_invalid_input_type():
    """Test that non-string inputs raise ValueError."""
    with pytest.raises(ValueError, match="SKU must be a string"):
        sku_sample_translator(123)
    with pytest.raises(ValueError, match="SKU must be a string"):
        sku_sample_translator(None)


def test_invalid_prefix():
    """Test that SKUs not starting with 'OLD-' raise ValueError."""
    with pytest.raises(ValueError, match="SKU must start with 'OLD-'"):
        sku_sample_translator("NEW-ABC-1234")
    with pytest.raises(ValueError, match="SKU must start with 'OLD-'"):
        sku_sample_translator("XXX-ABC-1234")


def test_invalid_format():
    """Test various invalid SKU formats."""
    invalid_skus = [
        "OLD-AB-1234",  # Too few letters
        "OLD-ABCD-1234",  # Too many letters
        "OLD-123-1234",  # Numbers instead of letters
        "OLD-ABC-123",  # Too few digits
        "OLD-ABC-12345",  # Too many digits
        "OLD-ABC-XXXX",  # Letters instead of numbers
        "OLD-A1C-1234",  # Mixed letters and numbers in middle
    ]

    for sku in invalid_skus:
        with pytest.raises(
            ValueError,
            match="SKU format must be 'OLD-XXX-YYYY' where X is a letter and Y is a digit",
        ):
            sku_sample_translator(sku)


now, lets run the tests

In [0]:
import pytest

# Run tests from test_sku_translator.py
pytest.main(["-v", "tools/test_sample_tool.py"])


Now, lets deploy the tool to Unity catalog.

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from tools.sample_tool import sku_sample_translator

client = DatabricksFunctionClient()
CATALOG = "ep"  # Change me!
SCHEMA = "cookbook_local_test"  # Change me if you want

# this will deploy the tool to UC, automatically setting the metadata in UC based on the tool's docstring & typing hints
tool_uc_info = client.create_python_function(func=sku_sample_translator, catalog=CATALOG, schema=SCHEMA, replace=True)

# the tool will deploy to a function in UC called `{catalog}.{schema}.{func}` where {func} is the name of the function
# Print the deployed Unity Catalog function name
print(f"Deployed Unity Catalog function name: {tool_uc_info.full_name}")


Now, wrap it into a UCTool that will be used by our Agent.  UC tool is just a Pydnatic base model that is serializable to YAML that will load the tool's metadata from UC and wrap it in a callable object.

In [0]:
from cookbook.tools.uc_tool import UCTool

# wrap the tool into a UCTool which can be passed to our Agent
translate_sku_tool = UCTool(uc_function_name=tool_uc_info.full_name)

Now, let's test the UC tool - the UCTool is a directly callable wrapper around the UC function, so it can be used just like a local function, but the output will be put into a dictionary with either the output in a 'value' key or an 'error' key if an error is raised.

when an error happens, the UC tool will also return an instruction prompt to show the agent how to think about handling the error.  this can be changed via the `error_prompt` parameter in the UCTool..


In [0]:
# successful call
translate_sku_tool(old_sku="OLD-XXX-1234")

In [0]:
# unsuccessful call
translate_sku_tool(old_sku="OxxLD-XXX-1234")

now, let's convert our pytests to work with the UC tool.  this requires a bit of transformation to the test code to account for the fact that the output is in a dictionary & exceptions are not raised directly.

In [0]:
%%writefile tools/test_sample_tool_uc.py
import pytest
from cookbook.tools.uc_tool import UCTool

# Load the function from the UCTool versus locally
@pytest.fixture
def uc_tool():
    """Fixture to translate a UC tool into a local function."""
    UC_FUNCTION_NAME = "ep.cookbook_local_test.sku_sample_translator"
    loaded_tool = UCTool(uc_function_name=UC_FUNCTION_NAME)
    return loaded_tool


# Note: The value will be post processed into the `value` key, so we must check the returned value there.
def test_valid_sku_translation(uc_tool):
    """Test successful SKU translation with valid input."""
    assert uc_tool(old_sku="OLD-ABC-1234")["value"] == "NEW-1234-ABC"
    assert uc_tool(old_sku="OLD-XYZ-0001")["value"] == "NEW-0001-XYZ"
    assert (
        uc_tool(old_sku="old-def-5678")["value"] == "NEW-5678-DEF"
    )  # Test case insensitivity


# Note: The value will be post processed into the `value` key, so we must check the returned value there.
def test_whitespace_handling(uc_tool):
    """Test that the function handles extra whitespace correctly."""
    assert uc_tool(old_sku="  OLD-ABC-1234  ")["value"] == "NEW-1234-ABC"
    assert uc_tool(old_sku="\tOLD-ABC-1234\n")["value"] == "NEW-1234-ABC"


# Note: the input validation happens BEFORE the function is called by Spark, so we will never get these exceptions from the function.
# Instead, we will get invalid parameters errors from Spark.
def test_invalid_input_type(uc_tool):
    """Test that non-string inputs raise ValueError."""
    assert (
        uc_tool(old_sku=123)["error"]["error_message"]
        == """Invalid parameters provided: {'old_sku': "Parameter old_sku should be of type STRING (corresponding python type <class 'str'>), but got <class 'int'>"}."""
    )
    assert (
        uc_tool(old_sku=None)["error"]["error_message"]
        == """Invalid parameters provided: {'old_sku': "Parameter old_sku should be of type STRING (corresponding python type <class 'str'>), but got <class 'NoneType'>"}."""
    )


# Note: The errors will be post processed into the `error_message` key inside the `error` top level key, so we must check for exceptions there.
def test_invalid_prefix(uc_tool):
    """Test that SKUs not starting with 'OLD-' raise ValueError."""
    assert (
        uc_tool(old_sku="NEW-ABC-1234")["error"]["error_message"]
        == "ValueError: SKU must start with 'OLD-'"
    )
    assert (
        uc_tool(old_sku="XXX-ABC-1234")["error"]["error_message"]
        == "ValueError: SKU must start with 'OLD-'"
    )


# Note: The errors will be post processed into the `error_message` key inside the `error` top level key, so we must check for exceptions there.
def test_invalid_format(uc_tool):
    """Test various invalid SKU formats."""
    invalid_skus = [
        "OLD-AB-1234",  # Too few letters
        "OLD-ABCD-1234",  # Too many letters
        "OLD-123-1234",  # Numbers instead of letters
        "OLD-ABC-123",  # Too few digits
        "OLD-ABC-12345",  # Too many digits
        "OLD-ABC-XXXX",  # Letters instead of numbers
        "OLD-A1C-1234",  # Mixed letters and numbers in middle
    ]

    expected_error = "ValueError: SKU format must be 'OLD-XXX-YYYY' where X is a letter and Y is a digit"
    for sku in invalid_skus:
        assert uc_tool(old_sku=sku)["error"]["error_message"] == expected_error


In [0]:
import pytest

# Run tests from test_sku_translator.py
pytest.main(["-v", "tools/test_sample_tool_uc.py"])


# Now, here's another example of a tool that executes python code.

In [0]:
%%writefile tools/code_exec.py
def python_exec(code: str) -> str:
    """
    Executes Python code in the sandboxed environment and returns its stdout. The runtime is stateless and you can not read output of the previous tool executions. i.e. No such variables "rows", "observation" defined. Calling another tool inside a Python code is NOT allowed.
    Use only standard python libraries and these python libraries: bleach, chardet, charset-normalizer, defusedxml, googleapis-common-protos, grpcio, grpcio-status, jmespath, joblib, numpy, packaging, pandas, patsy, protobuf, pyarrow, pyparsing, python-dateutil, pytz, scikit-learn, scipy, setuptools, six, threadpoolctl, webencodings, user-agents, cryptography.

    Args:
      code (str): Python code to execute. Remember to print the final result to stdout.

    Returns:
      str: The output of the executed code.
    """
    import sys
    from io import StringIO

    sys_stdout = sys.stdout
    redirected_output = StringIO()
    sys.stdout = redirected_output
    exec(code)
    sys.stdout = sys_stdout
    return redirected_output.getvalue()


In [0]:
from tools.code_exec import python_exec

python_exec("print('hello')")

Test it locally

In [0]:
%%writefile tools/test_code_exec.py

import pytest
from .code_exec import python_exec


def test_basic_arithmetic():
    code = """result = 2 + 2\nprint(result)"""
    assert python_exec(code).strip() == "4"


def test_multiple_lines():
    code = "x = 5\n" "y = 3\n" "result = x * y\n" "print(result)"
    assert python_exec(code).strip() == "15"


def test_multiple_prints():
    code = """print('first')\nprint('second')\nprint('third')\n"""
    expected = "first\nsecond\nthird\n"
    assert python_exec(code) == expected


def test_using_pandas():
    code = (
        "import pandas as pd\n"
        "data = {'col1': [1, 2], 'col2': [3, 4]}\n"
        "df = pd.DataFrame(data)\n"
        "print(df.shape)"
    )
    assert python_exec(code).strip() == "(2, 2)"


def test_using_numpy():
    code = "import numpy as np\n" "arr = np.array([1, 2, 3])\n" "print(arr.mean())"
    assert python_exec(code).strip() == "2.0"


def test_syntax_error():
    code = "if True\n" "    print('invalid syntax')"
    with pytest.raises(SyntaxError):
        python_exec(code)


def test_runtime_error():
    code = "x = 1 / 0\n" "print(x)"
    with pytest.raises(ZeroDivisionError):
        python_exec(code)


def test_undefined_variable():
    code = "print(undefined_variable)"
    with pytest.raises(NameError):
        python_exec(code)


def test_multiline_string_manipulation():
    code = "text = '''\n" "Hello\n" "World\n" "'''\n" "print(text.strip())"
    expected = "Hello\nWorld"
    assert python_exec(code).strip() == expected

# Will not fail locally, but will fail in UC.
# def test_unauthorized_flask():
#     code = "from flask import Flask\n" "app = Flask(__name__)\n" "print(app)"
#     with pytest.raises(ImportError):
#         python_exec(code)


def test_no_print_statement():
    code = "x = 42\n" "y = x * 2"
    assert python_exec(code) == ""


def test_calculation_without_print():
    code = "result = sum([1, 2, 3, 4, 5])\n" "squared = [x**2 for x in range(5)]"
    assert python_exec(code) == ""


def test_function_definition_without_call():
    code = "def add(a, b):\n" "    return a + b\n" "result = add(3, 4)"
    assert python_exec(code) == ""


def test_class_definition_without_instantiation():
    code = (
        "class Calculator:\n"
        "    def add(self, a, b):\n"
        "        return a + b\n"
        "calc = Calculator()"
    )
    assert python_exec(code) == ""


In [0]:
import pytest

# Run tests from test_sku_translator.py
pytest.main(["-v", "tools/test_code_exec.py"])



Deploy to UC

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from tools.code_exec import python_exec
from cookbook.tools.uc_tool import UCTool

client = DatabricksFunctionClient()
CATALOG = "ep"  # Change me!
SCHEMA = "cookbook_local_test"  # Change me if you want

# this will deploy the tool to UC, automatically setting the metadata in UC based on the tool's docstring & typing hints
python_exec_tool_uc_info = client.create_python_function(func=python_exec, catalog=CATALOG, schema=SCHEMA, replace=True)

# the tool will deploy to a function in UC called `{catalog}.{schema}.{func}` where {func} is the name of the function
# Print the deployed Unity Catalog function name
print(f"Deployed Unity Catalog function name: {python_exec_tool_uc_info.full_name}")



Test as UC Tool for the Agent

In [0]:
from cookbook.tools.uc_tool import UCTool


# wrap the tool into a UCTool which can be passed to our Agent
python_exec_tool = UCTool(uc_function_name=python_exec_tool_uc_info.full_name)

python_exec_tool(code="print('hello')")


New tests

In [0]:
%%writefile tools/test_code_exec_as_uc_tool.py

import pytest
from cookbook.tools.uc_tool import UCTool

CATALOG = "ep"
SCHEMA = "cookbook_local_test"


@pytest.fixture
def python_exec():
    """Fixture to provide the python_exec function from UCTool."""
    python_exec_tool = UCTool(uc_function_name=f"{CATALOG}.{SCHEMA}.python_exec")
    return python_exec_tool


def test_basic_arithmetic(python_exec):
    code = """result = 2 + 2\nprint(result)"""
    assert python_exec(code=code)["value"].strip() == "4"


def test_multiple_lines(python_exec):
    code = "x = 5\n" "y = 3\n" "result = x * y\n" "print(result)"
    assert python_exec(code=code)["value"].strip() == "15"


def test_multiple_prints(python_exec):
    code = """print('first')\nprint('second')\nprint('third')\n"""
    expected = "first\nsecond\nthird\n"
    assert python_exec(code=code)["value"] == expected


def test_using_pandas(python_exec):
    code = (
        "import pandas as pd\n"
        "data = {'col1': [1, 2], 'col2': [3, 4]}\n"
        "df = pd.DataFrame(data)\n"
        "print(df.shape)"
    )
    assert python_exec(code=code)["value"].strip() == "(2, 2)"


def test_using_numpy(python_exec):
    code = "import numpy as np\n" "arr = np.array([1, 2, 3])\n" "print(arr.mean())"
    assert python_exec(code=code)["value"].strip() == "2.0"


def test_syntax_error(python_exec):
    code = "if True\n" "    print('invalid syntax')"
    result = python_exec(code=code)
    assert "Syntax error at or near 'invalid'." in result["error"]["error_message"]


def test_runtime_error(python_exec):
    code = "x = 1 / 0\n" "print(x)"
    result = python_exec(code=code)
    assert "ZeroDivisionError" in result["error"]["error_message"]


def test_undefined_variable(python_exec):
    code = "print(undefined_variable)"
    result = python_exec(code=code)
    assert "NameError" in result["error"]["error_message"]


def test_multiline_string_manipulation(python_exec):
    code = "text = '''\n" "Hello\n" "World\n" "'''\n" "print(text.strip())"
    expected = "Hello\nWorld"
    assert python_exec(code=code)["value"].strip() == expected


def test_unauthorized_flask(python_exec):
    code = "from flask import Flask\n" "app = Flask(__name__)\n" "print(app)"
    result = python_exec(code=code)
    assert (
        "ModuleNotFoundError: No module named 'flask'"
        in result["error"]["error_message"]
    )


def test_no_print_statement(python_exec):
    code = "x = 42\n" "y = x * 2"
    assert python_exec(code=code)["value"] == ""


def test_calculation_without_print(python_exec):
    code = "result = sum([1, 2, 3, 4, 5])\n" "squared = [x**2 for x in range(5)]"
    assert python_exec(code=code)["value"] == ""


def test_function_definition_without_call(python_exec):
    code = "def add(a, b):\n" "    return a + b\n" "result = add(3, 4)"
    assert python_exec(code=code)["value"] == ""


def test_class_definition_without_instantiation(python_exec):
    code = (
        "class Calculator:\n"
        "    def add(self, a, b):\n"
        "        return a + b\n"
        "calc = Calculator()"
    )
    assert python_exec(code=code)["value"] == ""


In [0]:
import pytest

# Run tests from test_sku_translator.py
pytest.main(["-v", "tools/test_code_exec_as_uc_tool.py"])

